##### Libraries

In [ ]:
!pip install openpyxl
!pip install rouge_score
!pip install datasets

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=24ee19ba604b52b10b1325ed0d9834edef045e8a53968b77c042e54ddc463490
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Att

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, AdamW
from google.colab import drive
from datasets import load_metric
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


##### Data Pre-Processing

In [ ]:
data=pd.read_excel('/content/drive/MyDrive/dataset/all cerificates(updated).xlsx')
#/content/drive/MyDrive/dataset/all cerificates(updated).xlsx  (in personal mail)
#/content/drive/MyDrive/all certificates(updated).xlsx (other mail)
#/content/drive/MyDrive/all cerificates.xlsx     (in college mail)
data.head()

,Q,A,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,എന്താണ് വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ്?,"ഒരു വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ്, സാധാരണയാ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,കേരളത്തിൽ ഒരു വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റി...,റീജിയണൽ ട്രാൻസ്‌പോർട്ട് ഓഫീസിൽ (ആർടിഒ) അല്ലെങ്...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റിന് അപേക്ഷിക്കാ...,ആവശ്യമായ രേഖകളിൽ സാധാരണയായി ഉൾപ്പെടുന്നു:\n\nഅ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ് ലഭിക്കാൻ എത്ര...,RTO-യിലെ ജോലിഭാരവും ഡോക്യുമെൻ്റേഷൻ്റെ സമ്പൂർണ്...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ് ലഭിക്കുന്നതിന...,"അതെ, രജിസ്ട്രേഷൻ ചാർജുകൾ, റോഡ് ടാക്സ്, മറ്റ് ബ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.isna()

,Q,A,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
106,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
107,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
108,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
data=data[['Q','A']]
data.dropna(inplace=True)
data.isna()

,Q,A
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
105,False,False
106,False,False
107,False,False
108,False,False


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 109
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       100 non-null    object
 1   A       100 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [ ]:
data.shape

(100, 2)

In [ ]:
#separating questions and answers
questions=data['Q'].tolist()
answers=data['A'].tolist()

In [ ]:
#removing symbols from text
symbols = ['?', '!', '(', ')', '।', '-', '_', ',', '.', ';', ':', '\"', '\'', '[', ']', '{', '}', '“', '”', '‘', '’']
import re
pattern = '[' + re.escape(''.join(symbols)) + ']'
def remove_symbols(text, pattern):
    return re.sub(pattern, '', text)
for i in range(0,len(questions)):
  questions[i] = [remove_symbols(questions[i], pattern) ]
  answers[i] = [remove_symbols(answers[i], pattern)]

##### Load Models, Loss functions, Optimizers



In [ ]:
model_name = 'facebook/mbart-large-50'
#facebook/mbart-large-50-many-to-many-mmt
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fi

In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr =2e-5)

##### Tokenize Data

In [ ]:
max_length=128
def tokenize_data(value):
    input_ids = []
    attention_masks = []
    for item in value:
        tensor_values = tokenizer(item, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")
        input_ids.append(tensor_values['input_ids'])
        attention_masks.append(tensor_values['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

In [ ]:
#Converting question and answers into input ids and attention mask by calling the above function
train_question_ids, train_question_attention = tokenize_data(questions)
train_answer_ids, train_answer_attention = tokenize_data(answers)

##### Model Training

In [ ]:
batch_size = 8
train_dataset = TensorDataset(train_question_ids, train_question_attention, train_answer_ids, train_answer_attention)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        decoder_attention_mask = batch[3].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(train_loader)
    print(f'Train Loss: {avg_loss:.4f}')

Epoch 1/10
Train Loss: 0.1127
Epoch 2/10
Train Loss: 0.1041
Epoch 3/10
Train Loss: 0.0941
Epoch 4/10
Train Loss: 0.0788
Epoch 5/10
Train Loss: 0.0720
Epoch 6/10
Train Loss: 0.0731
Epoch 7/10
Train Loss: 0.0647
Epoch 8/10
Train Loss: 0.0585
Epoch 9/10
Train Loss: 0.0570
Epoch 10/10
Train Loss: 0.0474


In [ ]:
train_ques,val_question,train_ans,val_answers=train_test_split(questions,answers,test_size=0.2,random_state=42)
model.eval()
correct_predictions = 0
predicted_answer=[]
for i in range(len(val_question)):
    input_text = val_question[i]
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=max_length, padding='max_length', truncation=True).input_ids.to(device)
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
    predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_answer.append(predicted_text)
    print(f"Question: {input_text}")
    print(f"Predicted Answer: {predicted_answer[i]}")

##### Save Model

In [ ]:
model.save_pretrained('/content/drive/MyDrive/gov_model')
tokenizer.save_pretrained("/content/drive/MyDrive/gov_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/gov_model/tokenizer_config.json',
 '/content/drive/MyDrive/gov_model/special_tokens_map.json',
 '/content/drive/MyDrive/gov_model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/gov_model/added_tokens.json',
 '/content/drive/MyDrive/gov_model/tokenizer.json')